In [4]:
import glob
import sys
import os
import numpy as np
import xarray as xr
import datetime as dt
import gc
from tqdm import notebook
import datetime
import matplotlib
import tkinter as tk
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import netCDF4 as nc # Ocean Colour CCI file format

In [2]:
THREDDS_string='https://www.oceancolour.org//thredds/dodsC/cci/v5.0-release/sinusoidal/monthly/chlor_a'
monthly_files=[]
# Maximum range would be 1997 to 2020 - beware! this will take a while to run
for year in np.arange(2019,2020): 
    print(year)
    for month in np.arange(1, 13):
        filestring=THREDDS_string+'/%s/ESACCI-OC-L3S-CHLOR_A-MERGED-1M_MONTHLY_4km_SIN_PML_OCx-%s%02d-fv5.0.nc' %(year, year,month)
        monthly_files.append(filestring)
print("Monthly files start with {}".format(monthly_files[0]))

2019
Monthly files start with https://www.oceancolour.org//thredds/dodsC/cci/v5.0-release/sinusoidal/monthly/chlor_a/2019/ESACCI-OC-L3S-CHLOR_A-MERGED-1M_MONTHLY_4km_SIN_PML_OCx-201901-fv5.0.nc


In [3]:
Global_mean_chl=[]
dates=[]
for input_file in notebook.tqdm(monthly_files):
    try:
        data=xr.open_dataset(input_file)
        GMC=np.nanmean(np.log10(data['chlor_a']))
        dates.append(data.time.values)
        Global_mean_chl.append(pow(10.,GMC))
    except OSError as e:
        print(e)


  0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:
# Plot data
plt.plot(dates, Global_mean_chl)

In [ ]:
# Save array
dir = os.path.abspath('')  # Location of jupyter notebook
outsav = os.path.join(dir, "OC-CCI-chlor_a-time-series")
numpoints = len(dates)
print(numpoints, "date points")
print("Data: ",dates[0],Global_mean_chl[0])
# Save as numpy array - option to restore later
np.savez(outsav, dates, Global_mean_chl)

In [ ]:
# Save Chl data as CSV file
outfile = os.path.join(dir, "OC-CCI-chlor_a-time-series.csv")
if os.path.exists(outfile):
    os.remove(outfile)
ab1 = np.array(dates)
ab2 = np.array(Global_mean_chl)
ab = np.zeros(ab1.size, dtype=[('Date', 'U60'), ('Mean Chl', float)])
print(ab.shape, ab1.size, ab2.size)
ab['Date'] = np.transpose(ab1)
ab['Mean Chl'] = np.transpose(ab2)
np.savetxt(outfile, ab, newline='\n', fmt="%10s, %10.3f")